In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader    # dataset representation and loading
import torch.nn as nn                     # neural networks
import torch.nn.functional as F           # layers, activations and more
import torch.optim as optim               # optimizers e.g. gradient descent, ADAM, etc.
import numpy as np
import csv

: 

In [ ]:
device = None
if torch.cuda.is_available():
    print("Using the GPU")
    device = torch.device('cuda:0')
else:
    device = torch.device('cpu')
    print("NOTE: Using CPU")

: 

In [3]:
class CSVDataset(Dataset): 
    # 9 feats: trip count, num ops, num operands, num mem ops, num fops, num branches, est resmii, frequent path length, depth of loop

    def __init__(self, csv_path):
        csv_reader = csv.reader(open(csv_path, 'r'), delimiter=',')

        self.csv_lines = []
        for line in csv_reader:
            self.csv_lines.append(line)
    
    def __len__(self):
        return len(self.csv_lines)
    
    def __getitem__(self, idx):
        # line = [prob_name, f1, ... , f9, label, rank1, ..., rank8, time1, ..., time8]
        line = self.csv_lines[idx]
        X = torch.FloatTensor(line[1:10])
        y = line[10]
        return X, y

In [ ]:
class MLP(nn.Module):
    def __init__(self,input_size,hidden_dim_1,hidden_dim_2,num_classes,drop_prob=0.5):
        super(MLP, self).__init__()
        #Put GeneModel architecture here (WITHOUT THE FINAL LAYERS)
        self.hidden_dim_1=hidden_dim_1
        self.hidden_dim_2=hidden_dim_2
        self.num_classes=num_classes
        
        #self.flantten=torch.flatten()
        self.fc_1=nn.Linear(input_size,hidden_dim_1)
        self.fc_2=nn.Linear(hidden_dim_1,hidden_dim_2)
        self.fc_3=nn.Linear(hidden_dim_2,num_classes)

        self.relu = nn.LeakyReLU()
        self.dropout = nn.Dropout(p=drop_prob)

        self.loss_function = nn.CrossEntropyLoss()
    
    def forward(self, x):
        #Put GeneModel architecture here (WITHOUT THE FINAL LAYERS)
        x=x.float()
        #x=torch.flatten(x).float()
        x=self.fc_1(x)
        x=self.relu(x)
        x=self.fc_2(x)
        x=self.relu(x)
        x=self.fc_3(x)
        return x
    
    # def train(self, trainloader, optimizer, n_epochs = 30):
    #     train_loss = 0
    #     losses_epochs = []
    #     for epoch in range(n_epochs):
    #         print("Starting epoch: ", epoch)
    #         for X, y in train_batch:
    #             optimizer.zero_grad()

    #             outputs = self(X1, X2)
    #             loss = self.criterion(outputs, y)
    #             loss.backward()
    #             optimizer.step()
    #             train_loss += loss.item()
    #         print("train loss:", train_loss)
    #         losses_epochs.append(train_loss)
    #         train_loss = 0
    #     return losses_epochs

In [ ]:
mlp = MLP(8,8,8,8)
train_csv = "mock_train.csv"
val_csv = "mock_val.csv"

lr = 0.001
optimizer = optim.Adam(mlp.parameters(), lr=lr)
epochs = 30
batch_size = 32

# Dataloader
train_dataset = CSVDataset(train_csv)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataset = CSVDataset(val_csv)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

In [ ]:
for epoch in epochs:
    for X, y in train_loader:
        optimizer.zero_grad()
        pred = mlp(X)
        loss = mlp.loss_function(pred, y)
        loss.backward()
        optimizer.step()

    mlp.eval()
    for X, y in val_loader:
        pred = mlp(X)
        val_loss = mlp.loss_function(pred, y)
    
    